# TUGAS INDIVIDU
Nama: Ummul Qura
NIM : H071201017
Kelas : Teknologi Basis Data A

In [1]:
pip install mysqlclient

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install sqlalchemy

In [3]:
from sqlalchemy import Column, Integer, String, and_, or_, text, ForeignKey, DECIMAL, Date, Text
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, relationship, subqueryload, joinedload
from sqlalchemy.sql import func
from sqlalchemy.dialects.mysql import INTEGER, MEDIUMBLOB, MEDIUMTEXT, SMALLINT

# # Connect to Database

In [4]:
engine = create_engine('mysql://root@127.0.0.1/classicmodels', echo = True)

# # Declare Base Class

In [5]:
Base = declarative_base()
metadata = Base.metadata


class Office(Base):
    __tablename__ = 'offices'

    officeCode = Column(String(10), primary_key=True)
    city = Column(String(50), nullable=False)
    phone = Column(String(50), nullable=False)
    addressLine1 = Column(String(50), nullable=False)
    addressLine2 = Column(String(50))
    state = Column(String(50))
    country = Column(String(50), nullable=False)
    postalCode = Column(String(15), nullable=False)
    territory = Column(String(10), nullable=False)


class Productline(Base):
    __tablename__ = 'productlines'

    productLine = Column(String(50), primary_key=True)
    textDescription = Column(String(4000))
    htmlDescription = Column(MEDIUMTEXT)
    image = Column(MEDIUMBLOB)


class Employee(Base):
    __tablename__ = 'employees'

    employeeNumber = Column(INTEGER(11), primary_key=True)
    lastName = Column(String(50), nullable=False)
    firstName = Column(String(50), nullable=False)
    extension = Column(String(10), nullable=False)
    email = Column(String(100), nullable=False)
    officeCode = Column(ForeignKey('offices.officeCode'), nullable=False, index=True)
    reportsTo = Column(ForeignKey('employees.employeeNumber'), index=True)
    jobTitle = Column(String(50), nullable=False)

    office = relationship('Office')
    parent = relationship('Employee', remote_side=[employeeNumber])


class Product(Base):
    __tablename__ = 'products'

    productCode = Column(String(15), primary_key=True)
    productName = Column(String(70), nullable=False)
    productLine = Column(ForeignKey('productlines.productLine'), nullable=False, index=True)
    productScale = Column(String(10), nullable=False)
    productVendor = Column(String(50), nullable=False)
    productDescription = Column(Text, nullable=False)
    quantityInStock = Column(SMALLINT(6), nullable=False)
    buyPrice = Column(DECIMAL(10, 2), nullable=False)
    MSRP = Column(DECIMAL(10, 2), nullable=False)

    productline = relationship('Productline')


class Customer(Base):
    __tablename__ = 'customers'

    customerNumber = Column(INTEGER(11), primary_key=True)
    customerName = Column(String(50), nullable=False)
    contactLastName = Column(String(50), nullable=False)
    contactFirstName = Column(String(50), nullable=False)
    phone = Column(String(50), nullable=False)
    addressLine1 = Column(String(50), nullable=False)
    addressLine2 = Column(String(50))
    city = Column(String(50), nullable=False)
    state = Column(String(50))
    postalCode = Column(String(15))
    country = Column(String(50), nullable=False)
    salesRepEmployeeNumber = Column(ForeignKey('employees.employeeNumber'), index=True)
    creditLimit = Column(DECIMAL(10, 2))

    employee = relationship('Employee')


class Order(Base):
    __tablename__ = 'orders'

    orderNumber = Column(INTEGER(11), primary_key=True)
    orderDate = Column(Date, nullable=False)
    requiredDate = Column(Date, nullable=False)
    shippedDate = Column(Date)
    status = Column(String(15), nullable=False)
    comments = Column(Text)
    customerNumber = Column(ForeignKey('customers.customerNumber'), nullable=False, index=True)

    customer = relationship('Customer')


class Payment(Base):
    __tablename__ = 'payments'

    customerNumber = Column(ForeignKey('customers.customerNumber'), primary_key=True, nullable=False)
    checkNumber = Column(String(50), primary_key=True, nullable=False)
    paymentDate = Column(Date, nullable=False)
    amount = Column(DECIMAL(10, 2), nullable=False)

    customer = relationship('Customer')


class Orderdetail(Base):
    __tablename__ = 'orderdetails'

    orderNumber = Column(ForeignKey('orders.orderNumber'), primary_key=True, nullable=False)
    productCode = Column(ForeignKey('products.productCode'), primary_key=True, nullable=False, index=True)
    quantityOrdered = Column(INTEGER(11), nullable=False)
    priceEach = Column(DECIMAL(10, 2), nullable=False)
    orderLineNumber = Column(SMALLINT(6), nullable=False)

    order = relationship('Order')
    product = relationship('Product')
    
Base.metadata.create_all(engine)

2022-12-15 15:00:23,708 INFO sqlalchemy.engine.Engine SHOW VARIABLES LIKE 'sql_mode'
2022-12-15 15:00:23,709 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-12-15 15:00:23,734 INFO sqlalchemy.engine.Engine SHOW VARIABLES LIKE 'lower_case_table_names'
2022-12-15 15:00:23,735 INFO sqlalchemy.engine.Engine [generated in 0.00204s] ()
2022-12-15 15:00:23,739 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2022-12-15 15:00:23,740 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-12-15 15:00:23,746 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-12-15 15:00:23,749 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %s AND table_name = %s
2022-12-15 15:00:23,752 INFO sqlalchemy.engine.Engine [generated in 0.00438s] ('classicmodels', 'offices')
2022-12-15 15:00:23,765 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %s AND table_name = %s
2022-12-15 15:00:23,765 INFO sqlalchemy.engine.Engine [cached

# #Declare Session

In [6]:
Session = sessionmaker(bind = engine)
session = Session()

## 1. Prepare a list of offices sorted by country, state, city. 

In [7]:
result = session.query(Office)

for row in result.order_by(Office.country,  Office.state, Office.city):
    print ("[Country]:",row.country, "[State]:",row.state, "[City]:",row.city) 

2022-12-15 15:00:23,844 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-12-15 15:00:23,857 INFO sqlalchemy.engine.Engine SELECT offices.`officeCode` AS `offices_officeCode`, offices.city AS offices_city, offices.phone AS offices_phone, offices.`addressLine1` AS `offices_addressLine1`, offices.`addressLine2` AS `offices_addressLine2`, offices.state AS offices_state, offices.country AS offices_country, offices.`postalCode` AS `offices_postalCode`, offices.territory AS offices_territory 
FROM offices ORDER BY offices.country, offices.state, offices.city
2022-12-15 15:00:23,863 INFO sqlalchemy.engine.Engine [generated in 0.00551s] ()
[Country]: Australia [State]: None [City]: Sydney
[Country]: France [State]: None [City]: Paris
[Country]: Japan [State]: Chiyoda-Ku [City]: Tokyo
[Country]: UK [State]: None [City]: London
[Country]: USA [State]: CA [City]: San Francisco
[Country]: USA [State]: MA [City]: Boston
[Country]: USA [State]: NY [City]: NYC


## 2. How many employees are there in the company? 

In [8]:
result = session.query(Employee).count()

print ("Employees in the company is :", result) 

2022-12-15 15:00:23,911 INFO sqlalchemy.engine.Engine SELECT count(*) AS count_1 
FROM (SELECT employees.`employeeNumber` AS `employees_employeeNumber`, employees.`lastName` AS `employees_lastName`, employees.`firstName` AS `employees_firstName`, employees.extension AS employees_extension, employees.email AS employees_email, employees.`officeCode` AS `employees_officeCode`, employees.`reportsTo` AS `employees_reportsTo`, employees.`jobTitle` AS `employees_jobTitle` 
FROM employees) AS anon_1
2022-12-15 15:00:23,914 INFO sqlalchemy.engine.Engine [generated in 0.00344s] ()
Employees in the company is : 23


## 3. What is the total of payments received

In [9]:
result = session.query(func.sum(Payment.amount))

for row in result.all():
    print("Total of Payments Received is :", row[0])

2022-12-15 15:00:23,940 INFO sqlalchemy.engine.Engine SELECT sum(payments.amount) AS sum_1 
FROM payments
2022-12-15 15:00:23,941 INFO sqlalchemy.engine.Engine [generated in 0.00109s] ()
Total of Payments Received is : 8853839.23


## 4. List the product lines that contain 'Cars'. 

In [10]:
result = session.query(Productline).filter(Productline.productLine.like('%Cars%'))

for row in result:
   print ("[Product Line]:",row.productLine)

2022-12-15 15:00:23,970 INFO sqlalchemy.engine.Engine SELECT productlines.`productLine` AS `productlines_productLine`, productlines.`textDescription` AS `productlines_textDescription`, productlines.`htmlDescription` AS `productlines_htmlDescription`, productlines.image AS productlines_image 
FROM productlines 
WHERE productlines.`productLine` LIKE %s
2022-12-15 15:00:23,972 INFO sqlalchemy.engine.Engine [generated in 0.00326s] ('%Cars%',)
[Product Line]: Classic Cars
[Product Line]: Vintage Cars


## 5. Report total payments for October 28, 2004. 


In [11]:
result = session.query(Payment).filter_by(paymentDate='2004-10-28')

for row in result:
    print("Total Payments for October 28, 2004 : ",row.amount)

2022-12-15 15:00:24,000 INFO sqlalchemy.engine.Engine SELECT payments.`customerNumber` AS `payments_customerNumber`, payments.`checkNumber` AS `payments_checkNumber`, payments.`paymentDate` AS `payments_paymentDate`, payments.amount AS payments_amount 
FROM payments 
WHERE payments.`paymentDate` = %s
2022-12-15 15:00:24,002 INFO sqlalchemy.engine.Engine [generated in 0.00301s] ('2004-10-28',)
Total Payments for October 28, 2004 :  47411.33


## 6. Report those payments greater than 100.000.


In [12]:
result = session.query(Payment).filter(Payment.amount > 100000)

for row in result:
   print ("[Customer Number]:",row.customerNumber,"[Check Number]:",row.checkNumber,"[Payment Date]:",row.paymentDate,"[Amount]:",row.amount)

2022-12-15 15:00:24,032 INFO sqlalchemy.engine.Engine SELECT payments.`customerNumber` AS `payments_customerNumber`, payments.`checkNumber` AS `payments_checkNumber`, payments.`paymentDate` AS `payments_paymentDate`, payments.amount AS payments_amount 
FROM payments 
WHERE payments.amount > %s
2022-12-15 15:00:24,035 INFO sqlalchemy.engine.Engine [generated in 0.00332s] (100000,)
[Customer Number]: 124 [Check Number]: AE215433 [Payment Date]: 2005-03-05 [Amount]: 101244.59
[Customer Number]: 124 [Check Number]: KI131716 [Payment Date]: 2003-08-15 [Amount]: 111654.40
[Customer Number]: 141 [Check Number]: ID10962 [Payment Date]: 2004-12-31 [Amount]: 116208.40
[Customer Number]: 141 [Check Number]: JE105477 [Payment Date]: 2005-03-18 [Amount]: 120166.58
[Customer Number]: 148 [Check Number]: KM172879 [Payment Date]: 2003-12-26 [Amount]: 105743.00


## 7. List the products in each product line.


In [13]:
result = session.query(Product).group_by(Product.productLine, Product.productName)

for row in result :
    print("[Product Name]:", row.productName, "[Product Line]:", row.productLine)

2022-12-15 15:00:24,061 INFO sqlalchemy.engine.Engine SELECT products.`productCode` AS `products_productCode`, products.`productName` AS `products_productName`, products.`productLine` AS `products_productLine`, products.`productScale` AS `products_productScale`, products.`productVendor` AS `products_productVendor`, products.`productDescription` AS `products_productDescription`, products.`quantityInStock` AS `products_quantityInStock`, products.`buyPrice` AS `products_buyPrice`, products.`MSRP` AS `products_MSRP` 
FROM products GROUP BY products.`productLine`, products.`productName`
2022-12-15 15:00:24,062 INFO sqlalchemy.engine.Engine [generated in 0.00111s] ()
[Product Name]: 1948 Porsche 356-A Roadster [Product Line]: Classic Cars
[Product Name]: 1948 Porsche Type 356 Roadster [Product Line]: Classic Cars
[Product Name]: 1949 Jaguar XK 120 [Product Line]: Classic Cars
[Product Name]: 1952 Alpine Renault 1300 [Product Line]: Classic Cars
[Product Name]: 1952 Citroen-15CV [Product Line

## 8. How many products in each product line?


In [14]:
result = session.query(Product.productLine,func.count(Product.productLine)).group_by(Product.productLine)

for row in result:
    print(row)

2022-12-15 15:00:24,122 INFO sqlalchemy.engine.Engine SELECT products.`productLine` AS `products_productLine`, count(products.`productLine`) AS count_1 
FROM products GROUP BY products.`productLine`
2022-12-15 15:00:24,124 INFO sqlalchemy.engine.Engine [generated in 0.00175s] ()
('Classic Cars', 38)
('Motorcycles', 13)
('Planes', 12)
('Ships', 9)
('Trains', 3)
('Trucks and Buses', 11)
('Vintage Cars', 24)


## 9. What is the minimum payment received?


In [15]:
result = session.query(func.min(Payment.amount))

for row in result:
    print("Minimum Payment Received: ", row[0])

2022-12-15 15:00:24,137 INFO sqlalchemy.engine.Engine SELECT min(payments.amount) AS min_1 
FROM payments
2022-12-15 15:00:24,138 INFO sqlalchemy.engine.Engine [generated in 0.00127s] ()
Minimum Payment Received:  615.45


## 10. List all payments greater than twice the average payment.


In [16]:
result = session.query(func.round(func.avg(Payment.amount)*2))
all_payment = session.query(Payment).filter(Payment.amount >= result)

for row in all_payment:
    print("[Customer Number]:",row.customerNumber,"[Payment]:",row.amount)

2022-12-15 15:00:24,180 INFO sqlalchemy.engine.Engine SELECT payments.`customerNumber` AS `payments_customerNumber`, payments.`checkNumber` AS `payments_checkNumber`, payments.`paymentDate` AS `payments_paymentDate`, payments.amount AS payments_amount 
FROM payments 
WHERE payments.amount >= (SELECT round(avg(payments.amount) * %s) AS round_1 
FROM payments)
2022-12-15 15:00:24,181 INFO sqlalchemy.engine.Engine [generated in 0.00099s] (2,)
[Customer Number]: 114 [Payment]: 82261.22
[Customer Number]: 124 [Payment]: 101244.59
[Customer Number]: 124 [Payment]: 85410.87
[Customer Number]: 124 [Payment]: 83598.04
[Customer Number]: 124 [Payment]: 111654.40
[Customer Number]: 141 [Payment]: 116208.40
[Customer Number]: 141 [Payment]: 65071.26
[Customer Number]: 141 [Payment]: 120166.58
[Customer Number]: 148 [Payment]: 105743.00
[Customer Number]: 167 [Payment]: 85024.46
[Customer Number]: 239 [Payment]: 80375.24
[Customer Number]: 321 [Payment]: 85559.12
[Customer Number]: 323 [Payment]: 7

C:\Users\ASUS\anaconda3\lib\site-packages\sqlalchemy\sql\coercions.py:307: SAWarning: implicitly coercing SELECT object to scalar subquery; please use the .scalar_subquery() method to produce a scalar subquery.
  util.warn(


## 11. What is the average percentage markup of the MSRP on buyPrice?


In [17]:
result = session.query(func.avg((Product.MSRP - Product.buyPrice)/Product.buyPrice *100)).scalar()

print("Average Percentage Markup of the MSRP on buy Price:", result)

2022-12-15 15:00:24,213 INFO sqlalchemy.engine.Engine SELECT avg(((products.`MSRP` - products.`buyPrice`) / products.`buyPrice`) * %s) AS avg_1 
FROM products
2022-12-15 15:00:24,223 INFO sqlalchemy.engine.Engine [generated in 0.01034s] (100,)
Average Percentage Markup of the MSRP on buy Price: 88.7023921713


## 12. How many distinct products does ClassicModels sell? 


In [18]:
result = session.query(Product).distinct().count()

print("Classic Models sell",result) 

2022-12-15 15:00:24,244 INFO sqlalchemy.engine.Engine SELECT count(*) AS count_1 
FROM (SELECT DISTINCT products.`productCode` AS `products_productCode`, products.`productName` AS `products_productName`, products.`productLine` AS `products_productLine`, products.`productScale` AS `products_productScale`, products.`productVendor` AS `products_productVendor`, products.`productDescription` AS `products_productDescription`, products.`quantityInStock` AS `products_quantityInStock`, products.`buyPrice` AS `products_buyPrice`, products.`MSRP` AS `products_MSRP` 
FROM products) AS anon_1
2022-12-15 15:00:24,246 INFO sqlalchemy.engine.Engine [generated in 0.00194s] ()
Classic Models sell 110


## 13. Report the name and city of customers who don't have sales representatives?


In [19]:
result = session.query(Customer).filter(Customer.salesRepEmployeeNumber.is_(None)).order_by(Customer.customerName)

for row in result:
    print("[Costumer Name]:",row.customerName, "[Costumer City]: ",row.city)

2022-12-15 15:00:24,279 INFO sqlalchemy.engine.Engine SELECT customers.`customerNumber` AS `customers_customerNumber`, customers.`customerName` AS `customers_customerName`, customers.`contactLastName` AS `customers_contactLastName`, customers.`contactFirstName` AS `customers_contactFirstName`, customers.phone AS customers_phone, customers.`addressLine1` AS `customers_addressLine1`, customers.`addressLine2` AS `customers_addressLine2`, customers.city AS customers_city, customers.state AS customers_state, customers.`postalCode` AS `customers_postalCode`, customers.country AS customers_country, customers.`salesRepEmployeeNumber` AS `customers_salesRepEmployeeNumber`, customers.`creditLimit` AS `customers_creditLimit` 
FROM customers 
WHERE customers.`salesRepEmployeeNumber` IS NULL ORDER BY customers.`customerName`
2022-12-15 15:00:24,281 INFO sqlalchemy.engine.Engine [generated in 0.00225s] ()
[Costumer Name]: ANG Resellers [Costumer City]:  Madrid
[Costumer Name]: Anton Designs, Ltd. [C

## 14. What are the names of executives with VP or Manager in their title? Use the CONCAT function to combine the employee's first name and last name into a single field for reporting 


In [20]:
result = session.query(func.concat(Employee.firstName," " ,Employee.lastName), Employee.jobTitle).filter(
    or_(Employee.jobTitle.contains('VP'), Employee.jobTitle.contains('Manager')))

for row in result:
    print(row)

2022-12-15 15:00:24,312 INFO sqlalchemy.engine.Engine SELECT concat(employees.`firstName`, %s, employees.`lastName`) AS concat_1, employees.`jobTitle` AS `employees_jobTitle` 
FROM employees 
WHERE (employees.`jobTitle` LIKE concat(concat('%%', %s), '%%')) OR (employees.`jobTitle` LIKE concat(concat('%%', %s), '%%'))
2022-12-15 15:00:24,313 INFO sqlalchemy.engine.Engine [generated in 0.00114s] (' ', 'VP', 'Manager')
('Mary Patterson', 'VP Sales')
('Jeff Firrelli', 'VP Marketing')
('William Patterson', 'Sales Manager (APAC)')
('Gerard Bondur', 'Sale Manager (EMEA)')
('Anthony Bow', 'Sales Manager (NA)')


## 15. Which orders have a value greater thanh 5000?

In [21]:
result = session.query(Orderdetail).filter(Orderdetail.quantityOrdered*Orderdetail.priceEach > 5000)

for row in result:
    print("[Order Number]:", row.orderNumber)

2022-12-15 15:00:24,339 INFO sqlalchemy.engine.Engine SELECT orderdetails.`orderNumber` AS `orderdetails_orderNumber`, orderdetails.`productCode` AS `orderdetails_productCode`, orderdetails.`quantityOrdered` AS `orderdetails_quantityOrdered`, orderdetails.`priceEach` AS `orderdetails_priceEach`, orderdetails.`orderLineNumber` AS `orderdetails_orderLineNumber` 
FROM orderdetails 
WHERE orderdetails.`quantityOrdered` * orderdetails.`priceEach` > %s
2022-12-15 15:00:24,341 INFO sqlalchemy.engine.Engine [generated in 0.00159s] (5000,)
[Order Number]: 10103
[Order Number]: 10103
[Order Number]: 10105
[Order Number]: 10105
[Order Number]: 10105
[Order Number]: 10108
[Order Number]: 10108
[Order Number]: 10108
[Order Number]: 10109
[Order Number]: 10109
[Order Number]: 10109
[Order Number]: 10110
[Order Number]: 10110
[Order Number]: 10110
[Order Number]: 10112
[Order Number]: 10114
[Order Number]: 10115
[Order Number]: 10115
[Order Number]: 10117
[Order Number]: 10117
[Order Number]: 10117
[